In [1]:
import run
import config

c:\Users\lilywhite\.conda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cfg = config.Config()

In [3]:
GRU = run.Runner(cfg, showExample=True)

TypeError: str2dict() takes 1 positional argument but 2 were given

In [ ]:
GRU.do(save_path=cfg.model_save_path, output_log=True)

In [ ]:
GRU.illust_trainning_curve(saveFig='./GRU_Trainning.png')

In [ ]:
GRU.illust_heatmap(saveFig='./Attention_Heatmap.png')

In [ ]:
GRU.test(save_path=cfg.result_save_path, output_log=True)